In [ ]:
import pandas as pd
import glob
import re
import os
from functools import reduce
import plotly.express as px

In [ ]:
dpath = "/Users/christinabrady/Documents/codebase/ds4a_women_project/processed_data/*.csv"
allfiles = glob.glob(dpath)
word_ct_files = [fl for fl in allfiles if "word_freqs" in fl]

In [ ]:
word_ct_list = []
for fl in word_ct_files:
    tmp = pd.read_csv(fl)
    tmp.columns = ["word", os.path.basename(fl).split("_")[0]]
    word_ct_list.append(tmp)


In [ ]:
word_cts = reduce(lambda  left,right: pd.merge(left,right,on=['word'],
                                            how='outer'), word_ct_list)

In [ ]:
### because I used sets in order to speed up the code
### the one - five columns don't add up to the all column
### Though not ideal, I think it will still work
### I'm going to create a new column which is the sum of one - five
### to use to create proportions

word_cts["one_five"] = word_cts.one + word_cts.two + word_cts.three + word_cts.four + word_cts.five
word_cts.head()

In [ ]:
## use the sum of one-five columns to create proportions
## take the range of the proportions and determine which terms to include
range_cols = ["one", "two", "three", "four", "five"]
word_cts["range"] = round((word_cts[range_cols].max(axis = 1) - word_cts[range_cols].min(axis = 1))/
                          word_cts[range_cols].max(axis = 1)*100)

In [ ]:
## what does that range look like?
fig2 = px.scatter(word_cts, 
                  x = "one_five", 
                  y = "range", 
                  title = "Distribution of range", 
                  hover_data = ["word"])
fig2.show()

In [ ]:
word_cts.sort_values("range", ascending = True)

In [ ]:
def plot_term(term):
    dat = word_cts[word_cts.word == term].melt()
    dat["variable"] = pd.Categorical(dat["variable"], ["one", "two", "three", "four", "five", "one_five", "all", "term", "range"])
    dat = dat.sort_values("variable")
    fig = px.histogram(dat[dat.variable.isin(["one", "two", "three", "four", "five", "one_five", "all"])], 
                      x = "variable", 
                      y = "value",
                      title = "Term: " + term + "; " + "Range: " + 
                       str(dat.loc[dat.variable == "range", ["value"]].values))
    fig.show()

In [ ]:
plot_term("amazing")